<a href="https://colab.research.google.com/github/manaswini294/DL-Assignment/blob/main/RESNET18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ensure TensorFlow 2.x is used
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, AveragePooling2D, Add
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

# Define convolution with batch normalization
def Conv2d_BN(x, nb_filter, kernel_size, padding='same', strides=(1,1), name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv2D(nb_filter, kernel_size, padding=padding, strides=strides, activation='relu', name=conv_name)(x)
    x = BatchNormalization(axis=3, name=bn_name)(x)
    return x

# Define Residual Block for ResNet18 (2 convolution layers)
def Residual_Block(input_model, nb_filter, kernel_size, strides=(1,1), with_conv_shortcut=False):
    x = Conv2d_BN(input_model, nb_filter=nb_filter, kernel_size=kernel_size, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size, padding='same')

    # Need convolution on shortcut for adding different channels
    if with_conv_shortcut:
        shortcut = Conv2d_BN(input_model, nb_filter=nb_filter, strides=strides, kernel_size=kernel_size)
        x = Add()([x, shortcut])
        return x
    else:
        x = Add()([x, input_model])
        return x

# Build ResNet18
def ResNet18(width, height, depth, classes):
    Img = Input(shape=(width, height, depth))

    # Initial convolution
    x = Conv2d_BN(Img, 64, (7,7), strides=(2,2), padding='same')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Residual conv2_x (output 56x56x64)
    x = Residual_Block(x, nb_filter=64, kernel_size=(3,3))
    x = Residual_Block(x, nb_filter=64, kernel_size=(3,3))

    # Residual conv3_x (output 28x28x128)
    x = Residual_Block(x, nb_filter=128, kernel_size=(3,3), strides=(2,2), with_conv_shortcut=True)
    x = Residual_Block(x, nb_filter=128, kernel_size=(3,3))

    # Residual conv4_x (output 14x14x256)
    x = Residual_Block(x, nb_filter=256, kernel_size=(3,3), strides=(2,2), with_conv_shortcut=True)
    x = Residual_Block(x, nb_filter=256, kernel_size=(3,3))

    # Residual conv5_x (output 7x7x512)
    x = Residual_Block(x, nb_filter=512, kernel_size=(3,3), strides=(2,2), with_conv_shortcut=True)
    x = Residual_Block(x, nb_filter=512, kernel_size=(3,3))

    # Global Average Pooling and output layer
    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)

    # Create model
    model = Model(inputs=Img, outputs=x)
    return model
ResNet18_model = ResNet18(224, 224, 3, 2)
ResNet18_model.summary()
# Use learning_rate instead of lr in Adam optimizer
ResNet18_model.compile(optimizer=Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_92 (Conv2D)        │ (None, 112, 112, 64)   │          9,472 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_92    │ (None, 112, 112, 64)   │            256 │ conv2d_92[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 56, 56, 64)     │              0 │ batch_normalization_9… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_93 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_93    │ (None, 56, 56, 64)     │            256 │ conv2d_93[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_94 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ batch_normalization_9… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_94    │ (None, 56, 56, 64)     │            256 │ conv2d_94[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_40 (Add)              │ (None, 56, 56, 64)     │              0 │ batch_normalization_9… │
│                           │                        │                │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_95 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ add_40[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_95    │ (None, 56, 56, 64)     │            256 │ conv2d_95[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_96 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ batch_normalization_9… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_96    │ (None, 56, 56, 64)     │            256 │ conv2d_96[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_41 (Add)              │ (None, 56, 56, 64)     │              0 │ batch_normalization_9… │
│                           │                        │                │ add_40[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_97 (Conv2D)        │ (None, 28, 28, 128)    │         73,856 │ add_41[0][0]           │
├──────────────────────

 Total params: 12,568,194 (47.94 MB)

 Trainable params: 12,558,594 (47.91 MB)

 Non-trainable params: 9,600 (37.50 KB)